# Yet to be Titled
## Introduction
Investing wisely requires a deep understanding of a company's financial health and valuation. This project aims to support my goal of becoming a more informed and intelligent investor by analysing the stock and financial data of companies. Using Benjamin Graham's value investing principles, I will assess whether companies are overvalued or undervalued, helping to identify potential investment opportunities. 

Benjamin Graham emphasized investing in stocks based on their intrinsic value, which reflects the true worth of a business derived from fundamentals like earnings and dividends. He advocated for having a margin of safety—buying stocks at prices significantly below their intrinsic value to mitigate risk. Graham drew a distinction between investing and speculating, emphasizing the importance of focusing on long-term fundamentals rather than short-term market trends. He also stressed the importance of stable earnings and consistent dividends as indicators of a company’s financial health and advised against overpaying for stocks, cautioning that even great companies can be poor investments if purchased at excessive prices.

Benjamin Graham’s principles of value investing have stood the test of time because they are rooted in rationality, discipline, and a deep understanding of the market. His timeless wisdom provides a solid foundation for building long-term wealth.

## Aim
**Primary Objective:**
- To identify undervalued and overvalued stocks using financial and valuation metrics based on Benjamin Graham's principles.

**Secondary Objectives:**
- Analyze financial data (e.g., earnings, book value, debt-to-equity ratio) of selected companies.
- Create a scoring system to evaluate companies based on Graham's criteria.
- Visualize findings for better decision-making.


## Deliverables
- A Jupyter Notebook with the complete analysis and visualizations.
- Key metrics for evaluating company valuation (e.g., P/E ratio, P/B ratio, intrinsic value estimates).
- A ranked list of companies based on their value investing potential.
- Visualizations showing trends and insights (e.g., scatter plots, heatmaps, dashboards).
- A summary report of findings and recommendations.

## Tools and Technologies
These are the tools, including libraries that will be used for this project:
- **Programming Language:** Python
- **Libraries:** Requests, Python-dotenv, Pandas, NumPy, Matplotlib, Seaborn, Plotly, Scikit-learn
- **Data Source APIs:** Alpha Vantage, Yahoo Finance
- **Development Environment:** Jupyter Notebook

---

### Data Acquisition and Preprocessing

I will begin by loading the required libraries for this project

In [4]:
import requests
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from dotenv import load_dotenv
import os

I wil load my API key and import data from Alpha Vantage using their API. 

Initially I will be starting with Apple stock and would obtain daily stock market data for up to the past 20 years.

In [5]:
load_dotenv()

# Load my API key into the varible
api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

symbol = "AAPL" 
url = f"https://www.alphavantage.co/query"

parameters = {
    "function": "TIME_SERIES_DAILY",
    "symbol": symbol,
    "apikey": api_key,
    "datatype": "json", 
    "outputsize": "full"
}

response = requests.get(url, params=parameters)

# Confirm the request was successful and extract the response
if response.status_code == 200:
    data = response.json()

In [6]:
# Subset the time series data from the response
time_series = data['Time Series (Daily)']

In [7]:
apple_stock = pd.DataFrame.from_dict(time_series, orient='index')
apple_stock.head()

,1. open,2. high,3. low,4. close,5. volume
2025-01-24,224.7800,225.6300,221.4100,222.7800,54697907
2025-01-23,224.7400,227.0300,222.3000,223.6600,60234760
2025-01-22,219.7900,224.1200,219.7900,223.8300,64126500
2025-01-21,224.0000,224.4200,219.3800,222.6400,98070429
2025-01-17,232.1150,232.2900,228.4800,229.9800,68488301


Now that I have Apple's stock data, I will need to get their cash flow, balance sheet, income statement, and earnings data. Getting this data one-by-one will be tiring and cumbersome, so I will create two functions to handle this. The first function will get the data required from the API, and the second function will format the obtained data into dataframes stored in their respective variables.

In [8]:
def get_data(function,symbol):
    """
        Fetch data from the Alpha Vantage API.
        
        Args:
            function (str): The type of data to request. Options are:
                            - "daily"
                            - "income"
                            - "balance"
                            - "cash"
                            - "eps"
        symbol (str or list): The ticker symbol(s) of the stock(s) to fetch data for. Can be a single symbol as a string or a list of symbols.
    
        Returns:
            dict: The JSON response from the Alpha Vantage API if the request is successful.
            str: Error message if the request fails or invalid function is provided.
        """
    # Define the base URL for the API
    url = f"https://www.alphavantage.co/query"

    # Fetch the API key from environment variables
    api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
    if not api_key:
        return "API key is missing. Please set the ALPHA_VANTAGE_API_KEY environment variable."

    
    # Map the functions to API parameters
    function_mapping = {
        "daily": "TIME_SERIES_DAILY",
        "income": "INCOME_STATEMENT",
        "balance": "BALANCE_SHEET",
        "cash": "CASH_FLOW",
        "eps": "EARNINGS"
    }

    # Check if the function is valid
    if function not in function_mapping:
        return f"Invalid function '{function}'. Valid options are: {', '.join(function_mapping.keys())}."

    # To handle a list of symbols
    if isinstance(symbol, list):
        # Store the data for each symbol
        results = {}
        for tick in symbol:
            parameters = {
                "function": function_mapping[function],
                "symbol": tick,
                "apikey": api_key
            }
            
            # Add additional parameters for specific functions
            if function == "daily":
                parameters.update({
                    "datatype": "json",
                    "outputsize": "full"
                })
        
            # Make the API request
            response = requests.get(url, params=parameters)
        
            # Handle the API response
            if response.status_code == 200:
                results[tick] = response.json()  # Store the response for this symbol
            else:
                results[tick] = f"Failed to fetch data. Status code: {response.status_code}. Reason: {response.reason}"

        return results  # Return all the results as a dictionary

    # Handle a single symbol
    else: 
        # Set up the parameters for the API request
        parameters = {
            "function": function_mapping[function],
            "symbol": symbol,
            "apikey": api_key
        }
    
        # Add additional parameters for specific functions
        if function == "daily":
            parameters.update({
                "datatype": "json",
                "outputsize": "full"
            })
    
        # Make the API request
        response = requests.get(url, params=parameters)
    
        # Handle the API response
        if response.status_code == 200:
            return response.json()
        else:
            return f"Failed to fetch data. Status code: {response.status_code}. Reason: {response.reason}"

In [9]:
def format_data(data,function,nested=False):
    """
    Format JSON data gotten from the Alpha Vantage API into a pandas DataFrame.
    
    Parameters:
    - data (dict): The raw JSON data from the Alpha Vantage API.
    - function (str): The type of data (e.g., 'daily', 'eps', 'income', 'balance', 'cash').
    - nested (bool): If False, expects a single time series. If True, expects a dictionary of time series.
    
    Returns:
    - pd.DataFrame or dict: A DataFrame or a dictionary of DataFrames depending on the 'nested' argument.

    Raises:
    - KeyError: If the expected keys are missing from the data.

    Examples:
        # Example for non-nested daily data
        format_data(data, function='daily', nested=False)

        # Example for nested EPS data
        format_data(data, function='eps', nested=True)
    """
    # Formats stock data
    if function == 'daily':
        if nested:
            stock_data = {}
            for key, values in data.items():
                time_series = values.get('Time Series (Daily)', None)
                if time_series is None:
                    raise KeyError(f"[{function}] Expected 'Time Series (Daily)' key not found in the nested data for {key}")
                stock_data[key] = pd.DataFrame.from_dict(time_series, orient='index')

            return stock_data
             
        else:
            time_series = data.get('Time Series (Daily)', None)
            if time_series is None:
                raise KeyError("Expected 'Time Series (Daily)' key not found in the data")
            stock_data = pd.DataFrame.from_dict(time_series, orient='index')
        
            return stock_data
    

    # Formats eps data
    elif function == 'eps':
        if nested:
            eps_data = {}
            for key, values in data.items():
                eps = values.get('annualEarnings', None)
                if eps is None:
                    raise KeyError(f"[{function}] Expected 'annualEarnings' key not found in the nested data for {key}")
                eps_data[key] = pd.DataFrame.from_dict(eps)

            return eps_data

        # Formats simple dictionaries
        else:
            eps = data.get('annualEarnings', None)
            if eps is None:
                raise KeyError("annualEarnings' key not found in the data")
            eps_data = pd.DataFrame.from_dict(eps)
        
            return eps_data

    # Formats income statement, cash flow, and balance sheet data
    else:
        # Formats nested dictionaries
        if nested:
            financial_data = {}
            for key, values in data.items():
                statement = values.get('annualReports', None)
                if statement is None:
                    raise KeyError(f"[{function}] Expected 'annualReports' key not found in the nested data for {key}")
                financial_data[key] = pd.DataFrame.from_dict(statement)

            return financial_data

        # Formats simple dictionaries
        else:
            statement = data.get('annualReports', None)
            if statement is None:
                raise KeyError("annualReports' key not found in the data")
            financial_data = pd.DataFrame.from_dict(statement)
        
            return financial_data

Now that I have created the functions, I will fetch Apple's income statement, balance sheet, cash flow, and earnings data using them

In [10]:
apple_eps = get_data('eps','AAPL')
apple_balance_sheet = get_data('balance', 'AAPL')
apple_income = get_data('income', 'AAPL')
apple_cash = get_data('cash', 'AAPL')

In [11]:
apple_eps = format_data(apple_eps,'eps')
apple_balance_sheet = format_data(apple_balance_sheet,'balance')
apple_income = format_data(apple_income, 'income')
apple_cash = format_data(apple_cash, 'cash')

---

### Data Cleaning

I will now begin cleaning my data to ensure it is ready for analysis. My first step will be to properly rename the columns. After that, I would check the dataframe to see if there are any null values and whether their datatypes are correct.

In [12]:
# Rename columns
apple_stock.rename(columns={
        "1. open": "Open",
        "2. high": "High",
        "3. low": "Low",
        "4. close": "Close",
        "5. volume": "Volume"
    }, inplace=True)

In [13]:
apple_stock.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6347 entries, 2025-01-24 to 1999-11-01
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Open    6347 non-null   object
 1   High    6347 non-null   object
 2   Low     6347 non-null   object
 3   Close   6347 non-null   object
 4   Volume  6347 non-null   object
dtypes: object(5)
memory usage: 297.5+ KB


In [14]:
apple_eps.head()

,fiscalDateEnding,reportedEPS
0,2024-12-31,0
1,2024-09-30,6.75
2,2023-09-30,6.12
3,2022-09-30,6.11
4,2021-09-30,5.62


In [15]:
apple_eps.rename(columns={
    'fiscalDateEnding': 'Fiscal Year End',
    'reportedEPS': 'Reported EPS'
    },
    inplace=True)

In [16]:
apple_eps.set_index('Fiscal Year End', inplace=True)

In [17]:
apple_eps.head()

,Reported EPS
Fiscal Year End,
2024-12-31,0
2024-09-30,6.75
2023-09-30,6.12
2022-09-30,6.11
2021-09-30,5.62


In [18]:
apple_balance_sheet.rename(columns={
    'fiscalDateEnding': 'Fiscal_Year_End',
    'reportedCurrency': 'Reported_Currency',
    'totalAssets': 'Total_Assets',
    'totalCurrentAssets': 'Total_Current_Assets',
    'cashAndCashEquivalentsAtCarryingValue': 'Cash_and_Cash_Equivalents',
    'cashAndShortTermInvestments': 'Cash_and_Short_Term_Investments',
    'inventory': 'Inventory',
    'currentNetReceivables': 'Current_Net_Receivables',
    'totalNonCurrentAssets': 'Total_Non_Current_Assets',
    'propertyPlantEquipment': 'Property_Plant_and_Equipment',
    'accumulatedDepreciationAmortizationPPE': 'Accumulated_Depreciation_on_PPE',
    'intangibleAssets': 'Intangible_Assets',
    'intangibleAssetsExcludingGoodwill': 'Intangible_Assets_Excl_Goodwill',
    'goodwill': 'Goodwill',
    'investments': 'Investments',
    'longTermInvestments': 'Long_Term_Investments',
    'shortTermInvestments': 'Short_Term_Investments',
    'otherCurrentAssets': 'Other_Current_Assets',
    'otherNonCurrentAssets': 'Other_Non_Current_Assets',
    'totalLiabilities': 'Total_Liabilities',
    'totalCurrentLiabilities': 'Total_Current_Liabilities',
    'currentAccountsPayable': 'Current_Accounts_Payable',
    'deferredRevenue': 'Deferred_Revenue',
    'currentDebt': 'Current_Debt',
    'shortTermDebt': 'Short_Term_Debt',
    'totalNonCurrentLiabilities': 'Total_Non_Current_Liabilities',
    'capitalLeaseObligations': 'Capital_Lease_Obligations',
    'longTermDebt': 'Long_Term_Debt',
    'currentLongTermDebt': 'Current_Long_Term_Debt',
    'longTermDebtNoncurrent': 'Non_Current_Long_Term_Debt',
    'shortLongTermDebtTotal': 'Total_Short_Long_Term_Debt',
    'otherCurrentLiabilities': 'Other_Current_Liabilities',
    'otherNonCurrentLiabilities': 'Other_Non_Current_Liabilities',
    'totalShareholderEquity': 'Total_Shareholder_Equity',
    'treasuryStock': 'Treasury_Stock',
    'retainedEarnings': 'Retained_Earnings',
    'commonStock': 'Common_Stock',
    'commonStockSharesOutstanding': 'Common_Stock_Shares_Outstanding'
},
    inplace=True)

In [19]:
apple_balance_sheet.set_index('Fiscal_Year_End', inplace=True)
apple_balance_sheet.head()

,Reported_Currency,Total_Assets,Total_Current_Assets,Cash_and_Cash_Equivalents,Cash_and_Short_Term_Investments,Inventory,Current_Net_Receivables,Total_Non_Current_Assets,Property_Plant_and_Equipment,Accumulated_Depreciation_on_PPE,...,Current_Long_Term_Debt,Non_Current_Long_Term_Debt,Total_Short_Long_Term_Debt,Other_Current_Liabilities,Other_Non_Current_Liabilities,Total_Shareholder_Equity,Treasury_Stock,Retained_Earnings,Common_Stock,Common_Stock_Shares_Outstanding
Fiscal_Year_End,,,,,,,,,,,,,,,,,,,,,
2024-09-30,USD,364980000000,152987000000,29943000000,65171000000,7286000000,66243000000,211993000000,45680000000,73448000000,...,10912000000,85750000000,106629000000,78304000000,45888000000,56950000000,None,-19154000000,83276000000,15116786000
2023-09-30,USD,352583000000,143566000000,29965000000,61555000000,6331000000,60985000000,209017000000,43715000000,70884000000,...,9822000000,95281000000,111088000000,58829000000,49848000000,62146000000,None,-214000000,73812000000,15550061000
2022-09-30,USD,352755000000,135405000000,23646000000,48304000000,4946000000,60932000000,217350000000,42117000000,72340000000,...,11128000000,98959000000,233256000000,60845000000,49142000000,50672000000,None,-3068000000,64849000000,15943425000
2021-09-30,USD,351002000000,134836000000,34940000000,62639000000,6580000000,51506000000,216166000000,39440000000,70283000000,...,9613000000,109106000000,241783000000,47493000000,53325000000,63090000000,None,5562000000,57365000000,16426786000
2020-09-30,USD,323888000000,143713000000,38016000000,90943000000,4061000000,37445000000,180175000000,36766000000,66760000000,...,8773000000,98667000000,216860000000,42684000000,54490000000,65339000000,None,14966000000,50779000000,16976763000


In [20]:
apple_cash.rename(columns={
    'fiscalDateEnding': 'Fiscal_Year_End',
    'reportedCurrency': 'Reported_Currency',
    'operatingCashflow': 'Operating_Cashflow',
    'paymentsForOperatingActivities': 'Payments_For_Operating_Activities',
    'proceedsFromOperatingActivities': 'Proceeds_From_Operating_Activities',
    'changeInOperatingLiabilities': 'Change_In_Operating_Liabilities',
    'changeInOperatingAssets': 'Change_In_Operating_Assets',
    'depreciationDepletionAndAmortization': 'Depreciation_Depletion_And_Amortization',
    'capitalExpenditures': 'Capital_Expenditures',
    'changeInReceivables': 'Change_In_Receivables',
    'changeInInventory': 'Change_In_Inventory',
    'profitLoss': 'Profit_Loss',
    'cashflowFromInvestment': 'Cashflow_From_Investment',
    'cashflowFromFinancing': 'Cashflow_From_Financing',
    'proceedsFromRepaymentsOfShortTermDebt': 'Proceeds_From_Repayments_Of_Short_Term_Debt',
    'paymentsForRepurchaseOfCommonStock': 'Payments_For_Repurchase_Of_Common_Stock',
    'paymentsForRepurchaseOfEquity': 'Payments_For_Repurchase_Of_Equity',
    'paymentsForRepurchaseOfPreferredStock': 'Payments_For_Repurchase_Of_Preferred_Stock',
    'dividendPayout': 'Dividend_Payout',
    'dividendPayoutCommonStock': 'Dividend_Payout_Common_Stock',
    'dividendPayoutPreferredStock': 'Dividend_Payout_Preferred_Stock',
    'proceedsFromIssuanceOfCommonStock': 'Proceeds_From_Issuance_Of_Common_Stock',
    'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet': 'Proceeds_From_Issuance_Of_Long_Term_Debt_And_Capital_Securities_Net',
    'proceedsFromIssuanceOfPreferredStock': 'Proceeds_From_Issuance_Of_Preferred_Stock',
    'proceedsFromRepurchaseOfEquity': 'Proceeds_From_Repurchase_Of_Equity',
    'proceedsFromSaleOfTreasuryStock': 'Proceeds_From_Sale_Of_Treasury_Stock',
    'changeInCashAndCashEquivalents': 'Change_In_Cash_And_Cash_Equivalents',
    'changeInExchangeRate': 'Change_In_Exchange_Rate',
    'netIncome': 'Net_Income'
    },
    inplace=True)

In [21]:
apple_cash.set_index('Fiscal_Year_End', inplace=True)
apple_cash.head()

,Reported_Currency,Operating_Cashflow,Payments_For_Operating_Activities,Proceeds_From_Operating_Activities,Change_In_Operating_Liabilities,Change_In_Operating_Assets,Depreciation_Depletion_And_Amortization,Capital_Expenditures,Change_In_Receivables,Change_In_Inventory,...,Dividend_Payout_Common_Stock,Dividend_Payout_Preferred_Stock,Proceeds_From_Issuance_Of_Common_Stock,Proceeds_From_Issuance_Of_Long_Term_Debt_And_Capital_Securities_Net,Proceeds_From_Issuance_Of_Preferred_Stock,Proceeds_From_Repurchase_Of_Equity,Proceeds_From_Sale_Of_Treasury_Stock,Change_In_Cash_And_Cash_Equivalents,Change_In_Exchange_Rate,Net_Income
Fiscal_Year_End,,,,,,,,,,,,,,,,,,,,,
2024-09-30,USD,118254000000,1900000000,None,21572000000,17921000000,11445000000,9447000000,5144000000,1046000000,...,15234000000,None,None,0,None,-94949000000,None,None,None,93736000000
2023-09-30,USD,110543000000,5703000000,None,1142000000,7719000000,11519000000,10959000000,417000000,1618000000,...,15025000000,None,None,5228000000,None,-77550000000,None,5760000000,None,96995000000
2022-09-30,USD,122151000000,4665000000,None,15558000000,14358000000,11104000000,10708000000,9343000000,-1484000000,...,14841000000,None,None,5465000000,None,-89402000000,None,-10952000000,None,99803000000
2021-09-30,USD,104038000000,4087000000,None,19801000000,24712000000,11284000000,11085000000,14028000000,2642000000,...,14467000000,None,1105000000,20393000000,None,-84866000000,None,-3860000000,None,94680000000
2020-09-30,USD,80674000000,4502000000,None,6935000000,1245000000,11056000000,7309000000,-8470000000,127000000,...,14081000000,None,880000000,16091000000,None,-71478000000,None,-10435000000,None,57411000000


In [22]:
apple_income.rename(columns={
    'fiscalDateEnding': 'Fiscal_Year_End',
    'reportedCurrency': 'Reported_Currency',
    'grossProfit': 'Gross_Profit',
    'totalRevenue': 'Total_Revenue',
    'costOfRevenue': 'Cost_Of_Revenue',
    'costofGoodsAndServicesSold': 'Cost_Of_Goods_And_Services_Sold',
    'operatingIncome': 'Operating_Income',
    'sellingGeneralAndAdministrative': 'Selling_General_And_Administrative',
    'researchAndDevelopment': 'Research_And_Development',
    'operatingExpenses': 'Operating_Expenses',
    'investmentIncomeNet': 'Investment_Income_Net',
    'netInterestIncome': 'Net_Interest_Income',
    'interestIncome': 'Interest_Income',
    'interestExpense': 'Interest_Expense',
    'nonInterestIncome': 'Non_Interest_Income',
    'otherNonOperatingIncome': 'Other_Non_Operating_Income',
    'depreciation': 'Depreciation',
    'depreciationAndAmortization': 'Depreciation_And_Amortization',
    'incomeBeforeTax': 'Income_Before_Tax',
    'incomeTaxExpense': 'Income_Tax_Expense',
    'interestAndDebtExpense': 'Interest_And_Debt_Expense',
    'netIncomeFromContinuingOperations': 'Net_Income_From_Continuing_Operations',
    'comprehensiveIncomeNetOfTax': 'Comprehensive_Income_Net_Of_Tax',
    'ebit': 'Ebit',
    'ebitda': 'Ebitda',
    'netIncome': 'Net_Income'
    },
    inplace=True)

In [23]:
apple_income.set_index('Fiscal_Year_End', inplace=True)
apple_income.head()

,Reported_Currency,Gross_Profit,Total_Revenue,Cost_Of_Revenue,Cost_Of_Goods_And_Services_Sold,Operating_Income,Selling_General_And_Administrative,Research_And_Development,Operating_Expenses,Investment_Income_Net,...,Depreciation,Depreciation_And_Amortization,Income_Before_Tax,Income_Tax_Expense,Interest_And_Debt_Expense,Net_Income_From_Continuing_Operations,Comprehensive_Income_Net_Of_Tax,Ebit,Ebitda,Net_Income
Fiscal_Year_End,,,,,,,,,,,,,,,,,,,,,
2024-09-30,USD,180683000000,391035000000,236449000000,210352000000,123216000000,26097000000,31370000000,57467000000,None,...,8200000000,11445000000,123485000000,29749000000,None,93736000000,98016000000,123216000000,134661000000,93736000000
2023-09-30,USD,169148000000,379352000000,239069000000,214137000000,114301000000,24932000000,29915000000,54847000000,3750000000,...,8500000000,11519000000,113736000000,16741000000,3933000000,96995000000,96652000000,117669000000,125820000000,96995000000
2022-09-30,USD,170782000000,391397000000,248640000000,223546000000,119437000000,25094000000,26251000000,51345000000,2825000000,...,8700000000,11104000000,119103000000,19300000000,2931000000,99803000000,88531000000,122034000000,130541000000,99803000000
2021-09-30,USD,152836000000,363172000000,234954000000,212981000000,108949000000,21973000000,21914000000,43887000000,2843000000,...,9500000000,11284000000,109207000000,14527000000,2645000000,94680000000,95249000000,111852000000,120233000000,94680000000
2020-09-30,USD,104956000000,271642000000,189475000000,169559000000,66288000000,19916000000,18752000000,38668000000,3763000000,...,9700000000,11056000000,67091000000,9680000000,2873000000,57411000000,57453000000,69964000000,77344000000,57411000000


Now that I have renamed the columns, I will move on to dropping the columns that won't be relevant in this analysis

In [25]:
apple_stock.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6347 entries, 2025-01-24 to 1999-11-01
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Open    6347 non-null   object
 1   High    6347 non-null   object
 2   Low     6347 non-null   object
 3   Close   6347 non-null   object
 4   Volume  6347 non-null   object
dtypes: object(5)
memory usage: 297.5+ KB


In [26]:
apple_stock.drop(columns=['Volume', 'High', 'Low'], inplace=True)

I dropped these columns because the focus of this analysis is on long-term valuation and fundamental analysis. 